## 获取验证码数据

In [1]:
import pywinauto
from pywinauto import clipboard
import time

In [2]:
app = pywinauto.Application().connect(path=r"C:\同花顺软件\同花顺\xiadan.exe",timeout=10)

C:\ProgramData\Anaconda3\lib\site-packages\pywinauto\application.py:1032: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  UserWarning)


In [3]:
main = app.top_window()

In [4]:
def get_left_menus_handle():
    while True:
        try:
            handle = main.window(
                control_id=129,
                class_name='SysTreeView32'
            )
            # sometime can't find handle ready, must retry
            handle.wait('ready', 2)
            return handle
        except:
            pass

In [ ]:
get_left_menus_handle().get_item(['查询[F4]', '资金股票']).click()

## 验证码识别步骤

In [ ]:
import os
from PIL import Image
import numpy as np
import uuid

In [2]:
def split_and_save(path):
    path = "captchas/" + path
    pix = np.array(Image.open(path).convert("L"))
    # threshold image
    pix = (pix < 200) * 255

    col_ranges = [
        [2, 2 + 9],
        [18, 18 + 9],
        [34, 34 + 9],
        [50, 50 + 9]
    ]
    # split and save
    for col_range in col_ranges:
        letter = pix[:, col_range[0]: col_range[1]]
        im = Image.fromarray(np.uint8(letter))
        save_path = "./letters/" + str(uuid.uuid4()) + ".png"
        im.save(save_path)


In [3]:
Image.open("captchas/00438176-1f34-431d-b6ef-36fcbb8d953c.png")

In [4]:
im_paths = filter(lambda fn: os.path.splitext(fn)[1].lower() == '.png',
                      os.listdir("captchas"))

for im_path in im_paths:
    print(im_path)
    split_and_save(im_path)

00438176-1f34-431d-b6ef-36fcbb8d953c.png
027f50b5-9cae-4e77-b9d7-dd97f85bb2c8.png
0346e8e3-64c7-490b-867a-4114b1a05888.png
04939206-43cb-47dc-aac7-0e4b36c08c4f.png
067b9dd2-449e-4868-b9a3-bc1213f25035.png
0793c96e-1c22-4a23-9339-147fe36c20c4.png
079ff89c-ac91-4960-be78-c505731d52e3.png
0addae1c-462a-40f4-8290-357f7dd4b24f.png
0c5c0ab9-b9b3-43a1-87d8-e7673e518a03.png
11ca34e7-f712-4b69-b453-c727ec547e44.png
11efc48a-71ab-4a69-bf85-1a5495473451.png
1240035b-2d22-4774-9e96-92de2aa5610f.png
12d45039-5470-4f89-8e14-fad346f97dc2.png
14e30f29-b785-4f9f-92b0-d9fb587dd2ce.png
17623df6-399f-457f-bf55-a4d1c98f63fc.png
1cc5d6d1-5c43-413b-a0d8-a755935c816b.png
1df3616b-a00c-4d36-9157-b43d28f80e8a.png
1f99fdaa-43cb-4871-ba35-b9e9ef53db47.png
20047167-d7e4-4cd0-b2e1-046721a21bac.png
212ae678-30f6-4e5d-8193-c5a562c49f07.png
24467888-f89b-41cd-8b2d-77683b7d247a.png
24c15218-9260-47d3-b827-d3f29c07d417.png
2651fb7b-521c-4ef8-9e58-d524a86db666.png
265952b8-1f3b-4b4f-8c6f-52f4b88117cf.png
2df76314-a787-4a

In [8]:
from sklearn.neighbors import KNeighborsClassifier
import pickle


def load_dataset():
    X = []
    y = []

    for i in range(60):
        path = "./dataset/%d%d.png" % (i / 6, i % 6 + 1)
        pix = np.array(Image.open(path).convert("L"))
        # print(pix.reshape(8*20).shape)
        X.append(pix.reshape(23*9))
        y.append(i/6)
    return np.array(X), np.array(y)

In [9]:
X, y = load_dataset()
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, y.astype(int))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [14]:
with open("captcha_model.pickle", "wb") as f:
    pickle.dump(knn, f)

In [15]:
def get_model():
    with open("captcha_model.pickle", "rb") as f:
        return pickle.load(f)


def captcha_recognize(path):
    model = get_model()
    
    pix = np.array(Image.open(path).convert("L"))
    # threshold image
    pix = (pix < 200) * 255

    col_ranges = [
        [2, 2 + 9],
        [18, 18 + 9],
        [34, 34 + 9],
        [50, 50 + 9]
    ]
    # split and save
    letters = []
    for col_range in col_ranges:
        letter = pix[:, col_range[0]: col_range[1]]
        letters.append(letter.reshape(9*23))
        
    res = model.predict(letters)
    return "".join([str(ch) for ch in res])

In [16]:
captcha_recognize("captchas/00438176-1f34-431d-b6ef-36fcbb8d953c.png")

'7317'